In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
df = pd.read_csv('out/compare/benchmark full route/numer_results.csv')
df['prc_red_eh'] = (-df['EH']+df['NC'])/(df['NC'])*100
df['prc_red_ehx'] = (-df['EHX']+df['NC'])/(df['NC'])*100
df['prc_red_eh'] = df['prc_red_eh'].round(decimals=2)
df['prc_red_ehx'] = df['prc_red_ehx'].round(decimals=2)
df[['parameter', 'prc_cancel', 'prc_red_eh', 'prc_red_ehx']]

,parameter,prc_cancel,prc_red_eh,prc_red_ehx
0,cv_h,0,-1.21,-1.96
1,cv_h,15,13.75,24.69
2,cv_h,30,17.56,18.24
3,wt,0,-2.52,-3.60
4,wt,15,7.91,13.82
5,wt,30,12.69,14.17
6,rbt,0,-1.54,0.38
7,rbt,15,13.94,23.45
8,rbt,30,14.92,18.71
9,lp,0,5.41,-2.70


In [15]:
df = pd.read_csv('out/compare/benchmark mid route/numer_results.csv')
df['prc_red_eh'] = (-df['EH']+df['NC'])/(df['NC'])*100
df['prc_red_ehx'] = (-df['EHX']+df['NC'])/(df['NC'])*100
df['prc_red_eh'] = df['prc_red_eh'].round(decimals=2)
df['prc_red_ehx'] = df['prc_red_ehx'].round(decimals=2)
df[['parameter', 'prc_cancel','prc_red_eh', 'prc_red_ehx']]

,parameter,prc_cancel,prc_red_eh,prc_red_ehx
0,cv_h,0,1.04,-2.17
1,cv_h,15,17.11,29.67
2,cv_h,30,21.20,20.28
3,wt,0,-1.13,-1.77
4,wt,15,7.00,12.52
5,wt,30,16.89,15.35
6,rbt,0,-1.70,-0.36
7,rbt,15,15.21,23.35
8,rbt,30,19.57,21.23
9,lp,0,0.00,3.57


In [16]:
df = pd.read_pickle('out/EHX_dispatch/0715-182401-trip_record_ob.pkl')
expressed_trips = df[df['expressed']==1]['trip_id'].unique()

In [17]:
tmp_trip = expressed_trips.tolist()[0]
sample_1 = df[(df['trip_id'] == tmp_trip) & (df['expressed'] == 1)].copy()
sample_2 = df[(df['trip_id'] == tmp_trip) & (df['expressed'] == 0)].copy()

In [21]:
sample_1[(sample_1['stop_sequence'] < 18)]

,bus_id,trip_id,stop_id,arr_sec,dep_sec,pax_load,ons,offs,denied,hold_time,skipped,schd_sec,stop_sequence,dist_traveled,expressed,replication
439,898403,911903030,386,24720.000000,24727.500000,3,3,0,0,0,0,24840.0,1,0,1,1
483,898403,911903030,17163,25284.704777,25292.204777,4,1,0,0,0,0,25532.0,17,10056,1,1


In [24]:
sample_2[(sample_2['stop_sequence'] == 17) | (sample_2['stop_sequence'] == 1)]

,bus_id,trip_id,stop_id,arr_sec,dep_sec,pax_load,ons,offs,denied,hold_time,skipped,schd_sec,stop_sequence,dist_traveled,expressed,replication
1811,898403,911903030,386,24834.714959,24834.714959,0,0,0,0,0,0,24840.0,1,0,0,2
1881,898403,911903030,17163,25426.488118,25426.488118,12,0,0,0,0,0,25532.0,17,10056,0,2
3169,898403,911903030,386,24791.250000,24791.250000,0,0,0,0,0,0,24840.0,1,0,0,3
3235,898403,911903030,17163,25390.540194,25390.540194,9,0,0,0,0,0,25532.0,17,10056,0,3
4479,898403,911903030,386,24954.554656,24954.554656,8,8,0,0,0,0,24840.0,1,0,0,4
4531,898403,911903030,17163,25889.409644,25898.208441,60,1,0,6,0,0,25532.0,17,10056,0,4
8721,898403,911903030,386,24803.699966,24803.699966,0,0,0,0,0,0,24840.0,1,0,0,7
8802,898403,911903030,17163,25347.119511,25355.565679,5,1,0,0,0,0,25532.0,17,10056,0,7
10523,898403,911903030,386,24753.750000,24753.750000,2,2,0,0,0,0,24840.0,1,0,0,8
10600,898403,911903030,17163,25358.359487,25365.859487,7,1,0,0,0,0,25532.0,17,10056,0,8


In [25]:
df_s1_0 = sample_1[sample_1['stop_sequence'] == 1]
df_s1_1 = sample_1[sample_1['stop_sequence'] == 17]
t1 = df_s1_1['arr_sec'].iloc[0] - df_s1_0['dep_sec'].iloc[0]
df_s2_0 = sample_2[(sample_2['stop_sequence'] == 1) & (sample_2['replication'] == 3)]
df_s2_1 = sample_2[(sample_2['stop_sequence'] == 17) & (sample_2['replication'] == 3)]
t2 = df_s2_1['arr_sec'].iloc[0] - df_s2_0['dep_sec'].iloc[0]

In [26]:
t1

557.2047769203491

In [27]:
t2

599.29019448693